# Module 4 Assesment: Product Demand Forecasts


# Read the dataset

In [ ]:
# Read the dataset into a Pandas Dataframe.
import pandas as pd
df = pd.read_csv("Historical Product Demand.csv")
df

,Product_Code,Warehouse,Product_Category,Date,Order_Demand
0,Product_0993,Whse_J,Category_028,2012/7/27,100
1,Product_0979,Whse_J,Category_028,2012/1/19,500
2,Product_0979,Whse_J,Category_028,2012/2/3,500
3,Product_0979,Whse_J,Category_028,2012/2/9,500
4,Product_0979,Whse_J,Category_028,2012/3/2,500
...,...,...,...,...,...
1048570,Product_1791,Whse_J,Category_006,2016/4/27,1000
1048571,Product_1974,Whse_J,Category_006,2016/4/27,1
1048572,Product_1787,Whse_J,Category_006,2016/4/28,2500
1048573,Product_0901,Whse_J,Category_023,2016/10/7,50


In [8]:
# Does the dataset include any missing values? 
before_drop = df.shape
print('Shape Before NA Drop: ', before_drop)
df.dropna
after_drop = df.shape
print("Shape After NA Drop: ", after_drop)
if before_drop == after_drop:
    print("No NA Values detected")
else:
    print("NA Values have been dropped")

Shape Before NA Drop:  (1048575, 5)
Shape After NA Drop:  (1048575, 5)
No NA Values detected


# Extract Features

In [9]:
df_features = df.drop(['Date', 'Warehouse'], axis = 1)
df_features
print(df_features['Order_Demand'])

0           100 
1           500 
2           500 
3           500 
4           500 
           ...  
1048570    1000 
1048571       1 
1048572    2500 
1048573      50 
1048574       4 
Name: Order_Demand, Length: 1048575, dtype: object


# Perform Preprocessing
Perform any needed pre-processing on the chosen features including: Scaling, Encoding, Dealing with Nan values.
Hint: Use only the preprocessing steps you think are useful.

In [10]:
# Encoding
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(df_features['Product_Code'])
df_features['Product_Code'] = encoder.transform(df_features['Product_Code'])
encoder.fit(df_features['Product_Category'])
df_features['Product_Category'] = encoder.transform(df_features['Product_Category'])
df_features

,Product_Code,Product_Category,Order_Demand
0,982,27,100
1,968,27,500
2,968,27,500
3,968,27,500
4,968,27,500
...,...,...,...
1048570,1780,5,1000
1048571,1962,5,1
1048572,1776,5,2500
1048573,890,22,50


In [11]:
# Scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_features['Order_Demand'] = df_features['Order_Demand'].str.replace('(', "")
df_features['Order_Demand'] = df_features['Order_Demand'].str.replace(')', "")
df_features['Order_Demand'] = df_features['Order_Demand'].astype('int64')

# Split the data

In [12]:
# 80% training set, 10% validation set, 10% test set
from sklearn.model_selection import train_test_split
encoded_y = df_features.iloc[:,2].values
print(encoded_y)
encoded_X = df_features.iloc[:, 0:2]
print(encoded_X)
X_train, X_test, y_train, y_test = train_test_split(encoded_X, encoded_y, test_size=0.1, train_size=0.2, random_state=0) # split the data to a test set by 20% using 0.2
X_validate, X_test, y_validate, y_test = train_test_split(X_test, y_test, test_size=0.1,train_size=0.5, random_state=0) # split the test set by %50 using 0.5 

[ 100  500  500 ... 2500   50    4]
         Product_Code  Product_Category
0                 982                27
1                 968                27
2                 968                27
3                 968                27
4                 968                27
...               ...               ...
1048570          1780                 5
1048571          1962                 5
1048572          1776                 5
1048573           890                22
1048574           703                 0

[1048575 rows x 2 columns]


# Training K-Nearest Neighbor (KNN) Regression
Use a KNN regressor model to train your data. Choose the best k for the KNN algorithm by trying different values and validating performance on the validation set.

In [13]:
from sklearn.neighbors import KNeighborsRegressor 
# Choose the best value of K 
scores = []
results = 0
best_score = 0
neighbors = range(1,20)

for i in neighbors:
    model = KNeighborsRegressor(n_neighbors=i).fit(X_train, y_train)
    results = model.score(X_test, y_test)
    scores.append(round(results,2))
    if results > best_score:
        best_score = results
        best_k = i
        best_model = model
print(scores)
print(best_k)

[-0.42, 0.06, 0.1, 0.11, 0.12, 0.13, 0.11, 0.1, 0.11, 0.12, 0.11, 0.11, 0.11, 0.11, 0.11, 0.11, 0.11, 0.11, 0.11]
6


In [16]:
# Print the R-Squared score of your final K-Nearest Neighbor (KNN) regressor
accuracy = best_model.score(X_validate, y_validate)
y_pred = best_model.predict(X_test)
print('The Best model has an accuracy of: ', round(accuracy,2))

The Best model has an accuracy of:  0.09


# Challenge Yourself
Repeat step 6 for a different regression modelling technique.

In [ ]:
# takes extremely long to run this cell
from sklearn.ensemble import RandomForestRegressor
modelRF = RandomForestRegressor(criterion = 'absolute_error', max_leaf_nodes=100).fit(X_train, y_train.ravel())
scoreRF = modelRF.score(X_test, y_test)
y_pred = modelRF.predict(X_test)
print(scoreRF)

In [17]:
from sklearn.metrics import mean_squared_error
print("Accuracy: ", accuracy)
print("MSE: " + str(mean_squared_error(y_test, y_pred)))

Accuracy:  0.09044992596699253
MSE: 1369841326.0391974
